# Activity Matching System

## Initial Setup and Imports
This cell allows file upload in Google Colab environment. You'll need to upload your Excel files here.

In [ ]:
from google.colab import files
uploaded = files.upload()

## Required Libraries Installation and Import
This cell:
- Installs required Python packages for Arabic and French text processing
- Imports necessary libraries for text processing and analysis
- Sets up NLTK and Arabic text processing tools

In [ ]:
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
!pip install camel-tools
!pip install python-bidi
!pip install arabic-reshaper
!pip install pyarabic
!pip install nltk
import nltk
nltk.download('punkt_tab')
import arabic_reshaper
from bidi.algorithm import get_display
from pyarabic.araby import strip_tashkeel, strip_tatweel
import camel_tools
from camel_tools.utils.normalize import normalize_alef_ar, normalize_teh_marbuta_ar
from camel_tools.utils.normalize import normalize_teh_marbuta_ar
from camel_tools.tokenizers.word import simple_word_tokenize
from camel_tools.disambig.mle import MLEDisambiguator
import re
import string

## BilingualPreprocessor Class Implementation
Key components:
- Handles bilingual text preprocessing (Arabic and French)
- Provides methods for:
  - Loading data from Excel files
  - Validating dataframes
  - Processing text in both languages
  - Detecting language automatically
  - Preparing data for matching

### Important Methods:
1. `load_data()`: Loads three Excel files (existing, forbidden, new activities)
2. `preprocess_french_text()`: Specific preprocessing for French text
3. `preprocess_arabic_text()`: Specific preprocessing for Arabic text
4. `detect_language()`: Automatically detects text language
5. `prepare_data_for_matching()`: Prepares all datasets for the matching process

In [ ]:
# Test function to make sure NLTK resources are available
def ensure_nltk_resources():
    try:
        stopwords.words('french')
    except LookupError:
        import nltk
        nltk.download('stopwords')
        nltk.download('punkt')

In [ ]:
class BilingualPreprocessor:
    def __init__(self):
        """Initialize preprocessing tools"""

        try:
            stopwords.words('french')
        except LookupError:
            import nltk
            nltk.download('stopwords')
            nltk.download('punkt')

        ensure_nltk_resources()
        self.french_stopwords = set(stopwords.words("french"))
        self.arabic_stopwords = set(stopwords.words("arabic"))

    def load_data(self, existing_activities_path, forbidden_activities_path, new_activities_path):
        """Load data from files"""
        try:
            existing_df = pd.read_excel(existing_activities_path)
            forbidden_df = pd.read_excel(forbidden_activities_path)
            new_df = pd.read_excel(new_activities_path)
            return existing_df, forbidden_df, new_df
        except Exception as e:
            raise Exception(f"Error loading data: {str(e)}")

    def validate_dataframes(self, existing_df, forbidden_df, new_df):
        """Print column names for debugging"""
        print("Existing columns:", existing_df.columns.tolist())
        print("Forbidden columns:", forbidden_df.columns.tolist())
        print("New columns:", new_df.columns.tolist())

    def detect_language(self, text):
        """Detect if text is Arabic or French"""
        if not isinstance(text, str):
            return 'fr'

        arabic_pattern = re.compile(r'[\u0600-\u06FF]')
        if arabic_pattern.search(text):
            return 'ar'
        return 'fr'

    def preprocess_text(self, text):
        """Preprocess text with automatic language detection"""
        if not isinstance(text, str):
            return ""
        text = text.lower()
        words = text.split()
        words = [word for word in words if word not in self.french_stopwords and word not in self.arabic_stopwords]
        return " ".join(words)

    def get_text_columns(self, df):
        """
        Get text columns from DataFrame based on content
        Returns list of column names that contain text
        """
        text_columns = []
        for col in df.columns:
            sample = df[col].dropna().iloc[0] if not df[col].empty else None
            if isinstance(sample, str):
                text_columns.append(col)
        return text_columns

    def preprocess_dataframe(self, df, df_type='new'):
        """
        Preprocess dataframe with automatic column detection
        """
        df_processed = df.copy()

        text_columns = self.get_text_columns(df)
        print(f"Processing text columns for {df_type}: {text_columns}")

        for col in text_columns:
            processed_col_name = f"{col}_processed"
            language_col_name = f"{col}_language"

            df_processed[processed_col_name] = df[col].apply(self.preprocess_text)
            df_processed[language_col_name] = df[col].apply(self.detect_language)

        return df_processed

    def prepare_data_for_matching(self, existing_df, forbidden_df, new_df):
        """Prepare all data for matching"""

        self.validate_dataframes(existing_df, forbidden_df, new_df)

        existing_processed = self.preprocess_dataframe(existing_df, 'existing')
        forbidden_processed = self.preprocess_dataframe(forbidden_df, 'forbidden')
        new_processed = self.preprocess_dataframe(new_df, 'new')

        for df, name in [(existing_processed, 'existing'),
                        (forbidden_processed, 'forbidden'),
                        (new_processed, 'new')]:
            empty_processed = df.select_dtypes(include=['object']).apply(
                lambda x: x.str.strip().str.len() == 0).any()
            if empty_processed.any():
                print(f"Warning: Some entries are empty after preprocessing in {name}")

        return existing_processed, forbidden_processed, new_processed

## Testing Preprocessor Implementation
This section:
- Tests the BilingualPreprocessor implementation
- Loads sample data
- Processes the data through the preprocessing pipeline

In [ ]:
!pip install openpyxl
preprocessor = BilingualPreprocessor()

existing_df, forbidden_df, new_df = preprocessor.load_data(
    '/content/existing.xlsx',
    '/content/forbidden.xlsx',
    '/content/train.xlsx'
)

existing_processed, forbidden_processed, new_processed = preprocessor.prepare_data_for_matching(
    existing_df,
    forbidden_df,
    new_df
)

print("Dataset of new suggested instances")
print(new_processed.info())

print("Dataset of forbidden instances")
print(forbidden_processed.info())

print("Dataset of existing instances")
print(existing_processed.info())

## OptimizedActivityMatcher Class Implementation
Features:
- Uses multiple similarity metrics (TF-IDF, Fuzzy, BERT embeddings)
- Processes activities in batches for efficiency
- Provides detailed matching results and scores
- Includes debug mode for detailed processing information


### Key Methods:
1. `calculate_field_statistics()`: Analyzes patterns in existing activities
2. `check_activity_validity()`: Validates new activities against field patterns
3. `calculate_batch_similarities()`: Computes similarity scores efficiently
4. `process_activities()`: Main processing pipeline for new activities


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
!pip install fuzzywuzzy
from fuzzywuzzy import fuzz
from sentence_transformers import SentenceTransformer
from collections import defaultdict

In [ ]:
class OptimizedActivityMatcher:
    def __init__(self, similarity_threshold=0.75, embedding_threshold=0.85, debug=False):
        self.similarity_threshold = similarity_threshold
        self.embedding_threshold = embedding_threshold
        self.tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 2), max_features=5000)
        self.bert_model = SentenceTransformer('paraphrase-multilingual-mpnet-base-v2')
        self.debug = debug

    def preprocess_text(self, text):
        """Preprocess text for better matching"""
        if not isinstance(text, str):
            print(f"Warning: received non-string text: {text}")
            return ""
        text = text.lower()
        text = re.sub(r'[^\w\s]', ' ', text)
        text = re.sub(r'\s+', ' ', text)
        return text.strip()


    def calculate_field_statistics(self, existing_df):
        """Calculate statistics for each field using vectorized operations"""
        field_stats = {}

        if self.debug:
            print(f"Calculating field statistics for {len(existing_df)} existing activities")
            print("Fields found:", existing_df['field'].unique())

        # Fit the TF-IDF vectorizer on all existing activities
        all_texts = [self.preprocess_text(text) for text in existing_df['name_activity']]
        self.tfidf_vectorizer.fit(all_texts)

        for field, group in existing_df.groupby('field'):
            processed_texts = [self.preprocess_text(text) for text in group['name_activity']]

            # Calculate TF-IDF and embeddings for the field
            tfidf_matrix = self.tfidf_vectorizer.transform(processed_texts)
            bert_embeddings = self.bert_model.encode(processed_texts, batch_size=64, show_progress_bar=False)

            combined_text = ' '.join(processed_texts)
            words = combined_text.split()

            field_stats[field] = {
                'count': len(group),
                'avg_length': np.mean([len(text) for text in processed_texts]),
                'common_words': pd.Series(words).value_counts().head(10).to_dict(),
                'embeddings': {
                    'tfidf': tfidf_matrix,
                    'bert': bert_embeddings
                }
            }

            if self.debug:
                print(f"\nField: {field}")
                print(f"Number of activities: {len(group)}")
                print(f"Average length: {field_stats[field]['avg_length']:.2f}")
                print("Common words:", list(field_stats[field]['common_words'].keys())[:5])

        return field_stats

    def check_activity_validity(self, activity, field_stats):
        """Check if activity follows patterns common in its field"""
        field = activity['domaine']

        if self.debug:
            print(f"\nChecking validity for activity: {activity['activity']}")
            print(f"Field: {field}")

        if field not in field_stats:
            if self.debug:
                print(f"Field {field} not found in field_stats")
            return True

        stats = field_stats[field]
        processed_activity = self.preprocess_text(activity['activity'])
        activity_len = len(processed_activity)

        if self.debug:
            print(f"Activity length: {activity_len}")
            print(f"Field average length: {stats['avg_length']}")

        # Length check
        if activity_len < stats['avg_length'] * 0.2 or activity_len > stats['avg_length'] * 3:
            if self.debug:
                print("Failed length check")
            return False

        # Word overlap check
        activity_words = set(processed_activity.split())
        common_words = set(stats['common_words'].keys())
        overlap = len(common_words.intersection(activity_words))

        if self.debug:
            print(f"Word overlap: {overlap}")
            print(f"Common words found: {common_words.intersection(activity_words)}")

        return overlap >= 1

    def process_activities(self, new_df, existing_df, forbidden_df):
        """Process new activities in optimized batches"""
        if self.debug:
            print(f"\nProcessing {len(new_df)} new activities")
            print(f"Existing activities: {len(existing_df)}")
            print(f"Forbidden activities: {len(forbidden_df)}")

        results = []
        field_stats = self.calculate_field_statistics(existing_df)

        # Group activities by field
        for field, group in new_df.groupby('domaine'):
            if self.debug:
                print(f"\nProcessing field: {field}")
                print(f"Number of activities in this field: {len(group)}")

            # Find corresponding field in existing activities
            matching_field = next((ef for ef in existing_df['field'].unique()
                                 if self.preprocess_text(ef) == self.preprocess_text(field)), None)

            field_activities = existing_df[existing_df['field'] == matching_field] if matching_field else pd.DataFrame()

            for _, activity in group.iterrows():
                processed_activity = self.preprocess_text(activity['activity_processed'])
                activity_details = {
                    'code_pro': activity['code_pro'],
                    'wilaya': activity['wilaya'],
                    'activity': activity['activity'],
                    'field': activity['domaine'],
                    'status': 'to_review',
                    'reasons': []
                }

                # Check validity
                if not self.check_activity_validity(activity, field_stats):
                    activity_details['status'] = 'declined'
                    activity_details['reasons'].append('Non-conformity detected')
                    results.append(activity_details)
                    continue

                # Check similarities if field activities exist
                if not field_activities.empty:
                    similarities, detailed_scores = self.calculate_batch_similarities(
                        [{'activity': processed_activity, 'domaine': field}],
                        field_activities,
                        field_stats
                    )

                    if self.debug:
                        print(f"\nActivity: {activity['activity']}")
                        print(f"Max similarity: {np.max(similarities):.3f}")

                    # Check for matches
                    matches = similarities[0] > self.similarity_threshold
                    if np.any(matches):
                        activity_details['status'] = 'declined'
                        activity_details['reasons'].append('Already exists')


                # Check forbidden activities
                if not forbidden_df.empty:
                    forbidden_similarities = self.calculate_forbidden_similarities(processed_activity, forbidden_df)

                    if np.any(forbidden_similarities > self.similarity_threshold):
                        activity_details['status'] = 'declined'
                        activity_details['reasons'].append('Non-conformity detected')

                results.append(activity_details)

        results_df = pd.DataFrame(results)
        if self.debug:
            print("\nFinal Results:")
            print(f"Total processed: {len(results_df)}")
            print(f"Rejected: {(results_df['status'] == 'rejected').sum()}")
            print(f"To review: {(results_df['status'] == 'to_review').sum()}")

        return results_df

    def calculate_forbidden_similarities(self, processed_activity, forbidden_df):
        """Calculate similarities between the activity and forbidden activities"""
        forbidden_texts_french = [self.preprocess_text(text) for text in forbidden_df['Activity_French']]
        forbidden_texts_arabic = [self.preprocess_text(text) for text in forbidden_df['Activity_Arabic']]

        # Calculate embeddings for the activity
        activity_embedding = self.bert_model.encode([processed_activity], batch_size=1, show_progress_bar=False)

        # Calculate embeddings for forbidden activities
        forbidden_embeddings_french = self.bert_model.encode(forbidden_texts_french, batch_size=64, show_progress_bar=False)
        forbidden_embeddings_arabic = self.bert_model.encode(forbidden_texts_arabic, batch_size=64, show_progress_bar=False)

        # Calculate semantic similarities
        semantic_scores_french = cosine_similarity(activity_embedding, forbidden_embeddings_french)[0]
        semantic_scores_arabic = cosine_similarity(activity_embedding, forbidden_embeddings_arabic)[0]

        # Combine scores
        forbidden_similarities = np.maximum(semantic_scores_french, semantic_scores_arabic)

        if self.debug:
            print(f"Max forbidden similarity: {np.max(forbidden_similarities):.3f}")

        return forbidden_similarities



    def calculate_batch_similarities(self, new_activities, field_activities, field_stats):
        """Calculate similarities for multiple activities at once"""
        if self.debug:
            print(f"\nCalculating similarities for {len(new_activities)} activities")

        new_texts = [self.preprocess_text(act['activity']) for act in new_activities]
        existing_texts = [self.preprocess_text(act) for act in field_activities['name_activity']]

        # Calculate embeddings for new activities
        new_embeddings = self.bert_model.encode(new_texts, batch_size=64, show_progress_bar=False)

        # Get precomputed embeddings for field activities
        field = new_activities[0]['domaine']
        field_embeddings = field_stats[field]['embeddings'] if field in field_stats else None

        if field_embeddings is None:
            if self.debug:
                print(f"No precomputed embeddings found for field: {field}")
            return np.zeros((len(new_activities), len(existing_texts))), {
                'tfidf': np.zeros((len(new_activities), len(existing_texts))),
                'fuzzy': np.zeros((len(new_activities), len(existing_texts))),
                'semantic': np.zeros((len(new_activities), len(existing_texts)))
            }

        # Calculate similarities in batch
        semantic_scores = cosine_similarity(new_embeddings, field_embeddings['bert'])

        # Calculate TF-IDF similarities using the same vectorizer
        new_tfidf = self.tfidf_vectorizer.transform(new_texts)
        tfidf_scores = cosine_similarity(new_tfidf, field_embeddings['tfidf'])

        # Calculate fuzzy scores for high-similarity matches only
        potential_matches = (semantic_scores > self.similarity_threshold * 0.8)
        fuzzy_scores = np.zeros_like(semantic_scores)

        for i, new_text in enumerate(new_texts):
            for j in np.where(potential_matches[i])[0]:
                fuzzy_scores[i, j] = fuzz.ratio(new_text, existing_texts[j]) / 100

        # Combine scores
        final_scores = 0.3 * tfidf_scores + 0.3 * fuzzy_scores + 0.4 * semantic_scores

        if self.debug:
            print(f"Max similarity scores - Semantic: {np.max(semantic_scores):.3f}, "
                  f"TF-IDF: {np.max(tfidf_scores):.3f}, Fuzzy: {np.max(fuzzy_scores):.3f}")

        return final_scores, {
            'tfidf': tfidf_scores,
            'fuzzy': fuzzy_scores,
            'semantic': semantic_scores
        }



## Testing Matcher Implementation
This section tests the matching system with processed data and displays sample results.

In [ ]:
matcher = OptimizedActivityMatcher(debug=False)
results = matcher.process_activities(new_processed, existing_processed, forbidden_processed)

# Print some sample results
print("\nSample Results:")
print(results[['activity', 'status', 'reasons']].head())

## Accuracy Computation
This function:
- Compares matcher predictions with original statuses
- Maps different status categories
- Calculates overall accuracy percentage

In [ ]:
def compute_accuracy(results_df, original_df):
    """
    Compute the accuracy of the OptimizedActivityMatcher by comparing its predictions
    with the original status.

    """

    status_mapping = {
        'a verifier': 'to_review',
        'validée': 'to_review',
        "N'est pas conforme": 'declined',
        'Deja existante': 'declined',
        'manque de precision': 'declined'
    }

    # Apply the mapping to the original DataFrame
    original_df['mapped_status'] = original_df['validation de la proposition '].map(status_mapping)

    # Merge the results with the original DataFrame on 'code_pro'
    merged_df = results_df.merge(original_df, on='code_pro')

    # Check if the predicted status matches the mapped original status
    correct_predictions = (merged_df['status'] == merged_df['mapped_status']).sum()
    total_predictions = len(merged_df)

    # Calculate accuracy
    accuracy = correct_predictions / total_predictions
    return accuracy

In [ ]:
# Compute accuracy
accuracy = compute_accuracy(results, new_df)
print(f"Accuracy: {accuracy * 100:.2f}%")

## Results Export
Export the results of the testing as an excel file.


In [ ]:
# Basic export
results.to_excel('results.xlsx', index=False)

## Usage Notes
1. Make sure to upload your Excel files before running the notebook
2. The system expects specific column names in your input files
3. Use debug=True when running the matcher to see detailed processing information
4. Check the exported Excel file for comprehensive results

## Data Requirements
Your Excel files should contain:
1. Existing activities file:
   - code_activity
   - name_activity
   - field
2. Forbidden activities file:
   - Activity_French
   - Activity_Arabic
3. New activities file:
   - code_pro
   - activity
   - domaine
